# Scrap data

### import necessary libraries

In [ ]:
from bs4 import BeautifulSoup, Tag
import requests
import re
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from datetime import datetime

### scrap the first page

In [ ]:
# function to extract html document from given url
def getHTMLdocument(url):
    # give a fake user agent to pass security
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    }
    
    # request for HTML document of given url
    response = requests.get(url, headers=headers)
      
    # response will be provided in JSON format
    return response.text
    
# assign required credentials
# assign URL
url_to_scrape = "https://selogeraumali.com/annonces-immobilieres"
  
# create document
html_document = getHTMLdocument(url_to_scrape)
  
# create soap object
soup = BeautifulSoup(html_document, 'html.parser')

In [ ]:
soup.prettify()

### get the number of pages

In [ ]:
number_of_pages = 1 # there is no display of pages when there is only one page

page_numbers = soup.find_all("a", class_="page-numbers")
if page_numbers:
    page_numbers_list = list(page_numbers)
    if len(page_numbers_list) > 1:
        last_page = page_numbers_list[-2]
        number_of_pages = int(last_page.get_text())

### get the links for all pages

In [ ]:
property_links = []
for page_number in tqdm(range(1, number_of_pages + 1)):
    document = getHTMLdocument(url_to_scrape + f"/page/{page_number}")
    page_soup = BeautifulSoup(html_document, 'html.parser')
    property_titles = page_soup.find_all("h2", class_="property-title")
    property_links.extend(
        [
            a["href"] for h2 in property_titles for a in h2.children
            if isinstance(a, Tag) and a.has_attr("href")
        ]
    )

In [ ]:
property_links

### Iterate through every link and get the property details

In [ ]:
description_text_sep = "\n"
location_field_prefix = "location"
info_field_prefix = "info"
agent_field_prefix = "agent"
property_records = []

for property_link in tqdm(property_links):
    property_records.append({})
    property_records[-1]["source"] = property_link

    document = getHTMLdocument(property_link)
    property_soup = BeautifulSoup(document, 'html.parser')
    
    # find property ad title
    page_title = property_soup.find("h1")
    if page_title:
        property_records[-1]["title"] = page_title.get_text()
    
    # find property status
    property_status_div = property_soup.find("div", class_="property-status")
    if property_status_div:
        property_status = property_status_div.find("span")
        if property_status:
            property_records[-1]["status"] = property_status.get_text()
        
    # find price
    property_price_div = property_soup.find("div", class_="property-price")
    if property_price_div:
        property_price = property_price_div.find("span")
        if property_price:
            property_records[-1]["price"] = property_price.get_text()
    
    # find description
    property_description = property_soup.find_all("div", class_="property-description")
    if property_description:
        property_description = property_description[0]
        description_parts = property_description.find_all("p", recursive=True)

        property_records[-1]["description"] = description_text_sep.join(
            [part.get_text() for part in description_parts]
        )

    # find location details
    property_location = property_soup.find_all("div", class_="single-property-element property-location")
    if property_location:
        property_location = property_location[0]
        location_fields = property_location.find_all("strong", recursive=True)
        for field in location_fields:
            field_value = field.find_next_sibling("span")
            if field_value:
                property_records[-1][
                    f"{location_field_prefix}-{field.get_text()}"
                ] = field_value.get_text()
                
    # find additional info
    property_info = property_soup.find("div", id="ere-overview")
    if property_info:
        info_fields = property_info.find_all("strong", recursive=True)
        for field in info_fields:
            field_value = field.find_next_sibling("span")
            if field_value:
                property_records[-1][
                    f"{info_field_prefix}-{field.get_text()}"
                ] = field_value.get_text()
    
    
    property_info = property_soup.find("div", id="ere-features")
    if property_info:
        info_fields = property_info.find_all("a", recursive=True)
        for field in info_fields:
            property_records[-1][
                f"{info_field_prefix}-{field.get_text()}"
            ] = True
    
    # find agent info
    property_agent_heading = property_soup.find("div", class_="agent-heading")
    if property_agent_heading:
        agent_name = property_agent_heading.find("a", recursive=True)
        if agent_name:
            property_records[-1]["agent_name"] = agent_name.get_text()
        agent_type = property_agent_heading.find("span", recursive=True)
        if agent_type:
            property_records[-1]["agent_type"] = agent_type.get_text()
    
    property_agent_mobile = property_soup.find("div", class_="agent-mobile")
    if property_agent_mobile:
        agent_mobile = property_agent_mobile.find("span", recursive=True)
        if agent_mobile:
            property_records[-1]["agent_mobile"] = agent_mobile.get_text()
    
    property_agent_email = property_soup.find("div", class_="agent-email")
    if property_agent_email:
        agent_email = property_agent_email.find("span", recursive=True)
        if agent_email:
            property_records[-1]["agent_email"] = agent_mobile.get_text()
    
    # find ratings info
    property_ratings_average = property_soup.find("span", class_="ratings-average")
    if property_ratings_average:
        property_records[-1]["ratings_average"] = property_ratings_average.get_text()
    
    property_ratings_count = property_soup.find("span", class_="ratings-count")
    if property_ratings_count:
        property_records[-1]["ratings_count"] = property_ratings_count.get_text()
    
    
    property_ratings_overall_rating = property_soup.find("div", class_="overall-rating")
    if property_ratings_overall_rating:
        property_ratings_details = property_ratings_overall_rating.find_all("li")
        if property_ratings_details:
            for ratings_detail in property_ratings_details:
                ratings_detail_label = ratings_detail.find("span", class_="label")
                if ratings_detail_label:
                    ratings_detail_pct = ratings_detail_label.find_next_sibling("span", class_="label")
                    if ratings_detail_pct:
                        property_records[-1][
                            f"ratings_{ratings_detail_label.get_text()}"
                        ] = ratings_detail_pct.get_text()
    
    property_date = property_soup.find("span", class_="property-date")
    if property_date:
        property_records[-1]["property_date"] = property_date.get_text()
    
    property_reviews_ul = property_soup.find("ul", class_="reviews-list")
    if property_reviews_ul:
        property_reviews = property_reviews_ul.find_all("li")
        if property_reviews:
            print(property_link)
            print(property_reviews)

In [ ]:
property_records

### Create pandas dataframe from records and save data as csv

In [ ]:
property_df = pd.DataFrame.from_records(property_records)

In [ ]:
property_df.to_csv("properties.csv", index=None)
# To Do: store the data in hadoop or in other file storage

# Process data

In [ ]:
property_df = pd.read_csv("properties.csv")

In [ ]:
property_df.head()

# Clean data

### clean price

In [ ]:
import numpy as np

property_df.loc[property_df["price"] == "Prix sur demande", "price"] = np.nan

In [ ]:
property_df.head()

In [ ]:
prices_without_dot = property_df.loc[~property_df["price"].isna(), "price"].str.replace(r"\.", "", regex=True)

In [ ]:
prices_without_dot

In [ ]:
property_df["currency"] = prices_without_dot.str.extract(r"[0-9]+ ([a-zA-Z]+) \/?")

In [ ]:
property_df["currency"] .value_counts()

In [ ]:
# Fcfa is the only currency so it's safe to fill all the column with it
property_df["currency"] = "Fcfa"

In [ ]:
property_df["info-Surface"] = property_df["info-Surface"].str.extract(r"([0-9]+) m2")
property_df["info-Surface du terrain"] = property_df["info-Surface du terrain"].str.extract(r"([0-9]+) m2")

In [ ]:
property_df["info-Annexe"] = property_df["info-Annexe"].str.extract(r"([0-9]+) chambre")

In [ ]:
property_df["ratings_count"] = property_df["ratings_count"].str.extract(r"([0-9]+) Avis").astype(int)
property_df["ratings_1"] = (property_df["ratings_1"].str.extract(r"([0-9]+) %").astype(int) / 100).astype(float)
property_df["ratings_2"] = (property_df["ratings_2"].str.extract(r"([0-9]+) %").astype(int) / 100).astype(float)
property_df["ratings_3"] = (property_df["ratings_3"].str.extract(r"([0-9]+) %").astype(int) / 100).astype(float)
property_df["ratings_4"] = (property_df["ratings_4"].str.extract(r"([0-9]+) %").astype(int) / 100).astype(float)
property_df["ratings_5"] = (property_df["ratings_5"].str.extract(r"([0-9]+) %").astype(int) / 100).astype(float)

In [ ]:
property_df["price"] = prices_without_dot.str.extract(r"([0-9]+)").astype(float)

In [ ]:
months_fr_to_en = {
    "janvier": "January",
    "février": "February",
    "mars": "March",
    "avril": "April",
    "mai": "May",
    "juin": "June",
    "juillet": "July",
    "août": "August",
    "septembre": "September",
    "octobre": "October",
    "novembre": "November",
    "décembre": "December"
}
property_df["property_date"] = property_df["property_date"].apply(
    lambda date: datetime.strptime(
        re.sub(
            r"([0-9]{1,2}) ([a-zA-zéâ]+) ([0-9]{4})",
            lambda m: f"{m.group(1)} {months_fr_to_en.get(m.group(2))} {m.group(3)}" if len(m.groups()) == 3 else np.nan,
            date.strip()
        ),
        '%d %B %Y'
    )
)

In [ ]:
property_df.head()

In [ ]:
property_df[property_df["info- Gardiennage"] == property_df["info- Service de gardiennage"]][["info- Gardiennage", "info- Service de gardiennage"]]

### clean column names

In [ ]:
list(property_df.columns)

In [ ]:
property_df.rename(
    columns={
         'location-Adresse:': 'location_adress',
         'location-Pays:': 'location_country',
         'location-Ville / Région:': 'location_city',
         'location-Quartier / Cercle:': 'location_district',
         'location-Voisinage:': 'location_neighborhood',
         'info-Référence': 'info_reference',
         "info-Type d'annonce": 'info_ad_type',
         "info-Statut de l'annonce": 'info_ad_status',
         'info-Pièces': 'info_rooms',
         'info-Chambres': 'info_bedrooms',
         'info-Salles de bains': 'info_bathrooms',
         'info-Garages': 'info_garages',
         'info-Salon': 'info_living_rooms',
         'info-Annexe': 'info_annex_bedrooms',
         'location-Code postal:': 'info_postal_code',
         'info-Année de construction': 'info_building_year',
         'info-Surface': 'info_surface',
         'info-Surface du terrain': 'info_land_surface',
         'info-Étiquette': 'info_tag',
         'info-Salle à manger': 'info_dining_room',
         'info-Cuisine': 'info_kitchen',
         'info-Balcon / Terasse': 'info_balcony',
         'info- Accès internet / Wifi': 'info_internet',
         'info- Armoires': 'info_wardrobe',
         'info- Bouilloire': 'info_kettle',
         'info- Chaises': 'info_chairs',
         'info- Chauffe-eau': 'info_water_heater',
         'info- Climatisation': 'info_air_conditioner',
         'info- Commodes': 'info_dressers',
         'info- Électricité prépayé': 'info_prepaid_electricity',
         'info- Fer à repasser': 'info_iron',
         'info- Four micro-ondes': 'info_microwave',
         'info- Groupe électrogène': 'info_generator',
         'info- Lit pour adulte': 'info_adult_bed',
         'info- Réfrigérateur': 'info_fridge',
         'info- Tables': 'info_tables',
         'info- Télévision': 'info_tv',
         'info- Ventilateur': 'info_fan',
         'info- Machine à laver': 'info_washing_machine',
         'info- Gardiennage': 'info_security',
         'info-Magasin': 'info_box',
         'info-Annexes': 'info_dependencies',
         'info- Électricité': 'info_electricity',
         'info-Jardin': 'info_garden',
         'info- Bibliothèque': 'info_library',
         'info- Buffets': 'info_buffets',
         'info- Chaines TV numérique': 'info_digital_tv_channels',
         'info- Coiffeuse': 'info_dressing_table',
         'info- Cuisine entièrement équipée': 'info_fully_equipped_kitchen',
         'info- Entièrement meublé': 'info_fully_furnished',
         'info-Piscine': 'info_swimming_pool',
    },
    inplace=True
)

In [ ]:
# There are duplicate columns, we should remove them
to_remove = ["info-Prix", 'info- piscine', 'info- Service de gardiennage']
property_df.drop(columns=to_remove, axis=1, inplace=True)

### Fill na

In [ ]:
missed_values_pct = (100 * property_df.isna().sum() / len(property_df)).sort_values(ascending=False)
missed_values_pct.sort_values(ascending=False)
missing_values_df = pd.DataFrame({"column": list(missed_values_pct.index), "missing_pct": missed_values_pct}).reset_index().drop("index", axis=1)
missing_values_df[missing_values_df["missing_pct"] > 0]

In [ ]:
property_df["info_swimming_pool"].fillna(False, inplace=True)
property_df["info_swimming_pool"] = property_df["info_swimming_pool"].astype(bool)

property_df["info_library"].fillna(False, inplace=True)
property_df["info_library"] = property_df["info_library"].astype(bool)

property_df["info_generator"].fillna(False, inplace=True)
property_df["info_generator"] = property_df["info_generator"].astype(bool)

property_df["info_adult_bed"].fillna(False, inplace=True)
property_df["info_adult_bed"] = property_df["info_adult_bed"].astype(bool)

property_df["info_kettle"].fillna(False, inplace=True)
property_df["info_kettle"] = property_df["info_kettle"].astype(bool)

property_df["info_postal_code"].fillna("", inplace=True)
property_df["info_postal_code"] = property_df["info_postal_code"].astype(str)

property_df["info_internet"].fillna(False, inplace=True)
property_df["info_internet"] = property_df["info_internet"].astype(bool)

property_df["info_security"].fillna(False, inplace=True)
property_df["info_security"] = property_df["info_security"].astype(bool)

property_df["info_buffets"].fillna(False, inplace=True)
property_df["info_buffets"] = property_df["info_buffets"].astype(bool)

property_df["info_digital_tv_channels"].fillna(False, inplace=True)
property_df["info_digital_tv_channels"] = property_df["info_digital_tv_channels"].astype(bool)

property_df["info_dressing_table"].fillna(False, inplace=True)
property_df["info_dressing_table"] = property_df["info_dressing_table"].astype(bool)

property_df["info_fully_equipped_kitchen"].fillna(False, inplace=True)
property_df["info_fully_equipped_kitchen"] = property_df["info_fully_equipped_kitchen"].astype(bool)

property_df["info_fully_furnished"].fillna(False, inplace=True)
property_df["info_fully_furnished"] = property_df["info_fully_furnished"].astype(bool)

property_df["info_iron"].fillna(False, inplace=True)
property_df["info_iron"] = property_df["info_iron"].astype(bool)

property_df["info_dressers"].fillna(False, inplace=True)
property_df["info_dressers"] = property_df["info_dressers"].astype(bool)

property_df["info_chairs"].fillna(False, inplace=True)
property_df["info_chairs"] = property_df["info_chairs"].astype(bool)

property_df["info_tables"].fillna(False, inplace=True)
property_df["info_tables"] = property_df["info_tables"].astype(bool)

property_df["info_microwave"].fillna(False, inplace=True)
property_df["info_microwave"] = property_df["info_microwave"].astype(bool)

property_df["info_fridge"].fillna(False, inplace=True)
property_df["info_fridge"] = property_df["info_fridge"].astype(bool)

property_df["info_chairs"].fillna(False, inplace=True)
property_df["info_chairs"] = property_df["info_chairs"].astype(bool)

property_df["info_tv"].fillna(False, inplace=True)
property_df["info_tv"] = property_df["info_tv"].astype(bool)

property_df["info_washing_machine"].fillna(False, inplace=True)
property_df["info_washing_machine"] = property_df["info_washing_machine"].astype(bool)

property_df["info_electricity"].fillna(False, inplace=True)
property_df["info_electricity"] = property_df["info_electricity"].astype(bool)

property_df["info_box"].fillna(False, inplace=True)
property_df["info_box"] = property_df["info_box"].astype(bool)

property_df["info_wardrobe"].fillna(False, inplace=True)
property_df["info_wardrobe"] = property_df["info_wardrobe"].astype(bool)

property_df["info_garden"].fillna(False, inplace=True)
property_df["info_garden"] = property_df["info_garden"].astype(bool)

property_df["info_dining_room"].fillna(False, inplace=True)
property_df["info_dining_room"] = property_df["info_dining_room"].astype(bool)

property_df["info_prepaid_electricity"].fillna(False, inplace=True)
property_df["info_prepaid_electricity"] = property_df["info_prepaid_electricity"].astype(bool)

property_df["info_air_conditioner"].fillna(False, inplace=True)
property_df["info_air_conditioner"] = property_df["info_air_conditioner"].astype(bool)

property_df["info_dependencies"].fillna(False, inplace=True)
property_df["info_dependencies"] = property_df["info_dependencies"].astype(bool)

property_df["info_balcony"].fillna(0, inplace=True)
property_df["info_balcony"] = property_df["info_balcony"].astype(int)

property_df["info_fan"].fillna(False, inplace=True)
property_df["info_fan"] = property_df["info_fan"].astype(bool)

property_df["info_water_heater"].fillna(False, inplace=True)
property_df["info_water_heater"] = property_df["info_water_heater"].astype(bool)

property_df["info_kitchen"].fillna(False, inplace=True)
property_df["info_kitchen"] = property_df["info_kitchen"].astype(bool)

property_df["location_neighborhood"].fillna("", inplace=True)
property_df["location_neighborhood"] = property_df["location_neighborhood"].astype(str)

property_df["info_garages"].fillna(0, inplace=True)
property_df["info_garages"] = property_df["info_garages"].astype(int)

property_df["info_tag"].fillna("", inplace=True)
property_df["info_tag"] = property_df["info_tag"].astype(str)

property_df["agent_mobile"].fillna("", inplace=True)
property_df["agent_mobile"] = property_df["agent_mobile"].astype(str)

property_df["info_building_year"] = property_df["info_building_year"].astype(float)
property_df["ratings_average"] = property_df["ratings_average"].astype(float)
property_df["info_surface"] = property_df["info_surface"].astype(float)
property_df["info_land_surface"] = property_df["info_land_surface"].astype(float)
property_df['info_annex_bedrooms'] = property_df['info_annex_bedrooms'].astype(float)

In [ ]:
missed_values_pct = (100 * property_df.isna().sum() / len(property_df)).sort_values(ascending=False)
missed_values_pct.sort_values(ascending=False)
missing_values_df = pd.DataFrame({"column": list(missed_values_pct.index), "missing_pct": missed_values_pct}).reset_index().drop("index", axis=1)
missing_values_df[missing_values_df["missing_pct"] > 0]

# Check data quality

In [ ]:
property_df.info()

### check data types

In [ ]:
expected_types = {
     'source': np.dtype('O'),
     'title': np.dtype('O'),
     'status': np.dtype('O'),
     'price': np.dtype('float64'),
     'description': np.dtype('O'),
     'location_adress': np.dtype('O'),
     'location_country': np.dtype('O'),
     'location_city': np.dtype('O'),
     'location_district': np.dtype('O'),
     'location_neighborhood': np.dtype('O'),
     'info_reference': np.dtype('int64'),
     'info_ad_type': np.dtype('O'),
     'info_ad_status': np.dtype('O'),
     'info_rooms': np.dtype('float64'),
     'info_bedrooms': np.dtype('float64'),
     'info_bathrooms': np.dtype('float64'),
     'info_garages': np.dtype('int64'),
     'info_living_rooms': np.dtype('float64'),
     'info_annex_bedrooms': np.dtype('float64'),
     'agent_name': np.dtype('O'),
     'agent_type': np.dtype('O'),
     'agent_mobile': np.dtype('O'),
     'agent_email': np.dtype('O'),
     'ratings_average': np.dtype('float64'),
     'ratings_count': np.dtype('int64'),
     'ratings_5': np.dtype('float64'),
     'ratings_4': np.dtype('float64'),
     'ratings_3': np.dtype('float64'),
     'ratings_2': np.dtype('float64'),
     'ratings_1': np.dtype('float64'),
     'property_date': np.dtype('<M8[ns]'),
     'info_postal_code': np.dtype('O'),
     'info_building_year': np.dtype('float64'),
     'info_surface': np.dtype('float64'),
     'info_land_surface': np.dtype('float64'),
     'info_tag': np.dtype('O'),
     'info_dining_room': np.dtype('bool'),
     'info_kitchen': np.dtype('bool'),
     'info_balcony': np.dtype('int64'),
     'info_internet': np.dtype('bool'),
     'info_wardrobe': np.dtype('bool'),
     'info_kettle': np.dtype('bool'),
     'info_chairs': np.dtype('bool'),
     'info_water_heater': np.dtype('bool'),
     'info_air_conditioner': np.dtype('bool'),
     'info_dressers': np.dtype('bool'),
     'info_prepaid_electricity': np.dtype('bool'),
     'info_iron': np.dtype('bool'),
     'info_microwave': np.dtype('bool'),
     'info_generator': np.dtype('bool'),
     'info_adult_bed': np.dtype('bool'),
     'info_fridge': np.dtype('bool'),
     'info_tables': np.dtype('bool'),
     'info_tv': np.dtype('bool'),
     'info_fan': np.dtype('bool'),
     'info_washing_machine': np.dtype('bool'),
     'info_security': np.dtype('bool'),
     'info_box': np.dtype('bool'),
     'info_dependencies': np.dtype('bool'),
     'info_electricity': np.dtype('bool'),
     'info_garden': np.dtype('bool'),
     'info_library': np.dtype('bool'),
     'info_buffets': np.dtype('bool'),
     'info_digital_tv_channels': np.dtype('bool'),
     'info_dressing_table': np.dtype('bool'),
     'info_fully_equipped_kitchen': np.dtype('bool'),
     'info_fully_furnished': np.dtype('bool'),
     'info_swimming_pool': np.dtype('bool'),
     'currency': np.dtype('O')
}
dtypes = property_df.dtypes.to_dict()
for column, column_type in expected_types.items():
    if column_type != dtypes[column]:
        print(f"column {column} should have type {column_type} but has instead type {dtypes[column]}")

# Store data

In [ ]:
property_df.to_csv("properties_cleaned.csv", index=None)

# Transform data

In [ ]:
property_df = pd.read_csv("properties_cleaned.csv")

# Visualize data